In [ ]:
# Importing deep learning 
from ultralytics import YOLO

# Other functionlaities 
import os 
import pandas as pd 
import numpy as np 
import math 

# Computer vision libs
import cv2 
from PIL import Image
from PIL.ExifTags import TAGS
import exifread

# Ploting 
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
# Loading the trained model 
model = YOLO('../runs/segment/train15/weights/last.pt')

In [ ]:
# Defining the path to the image 
path_to_img = 'input/test_image.JPG'
base_name = os.path.basename(path_to_img)

# Reading and ploting the image 
img = cv2.imread(path_to_img)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

## Predicting pixels 

Once the model is loaded, we can extract the segments using the model object we just loaded. 

In [ ]:
# Applying the model 
results = model(path_to_img)

In [ ]:
# Extracting the image shape 
img_w, img_h = img.shape[1], img.shape[0]

# Extracting the segments 
segments = results[0].masks.xyn
print(f"Number of segments: {len(segments)}")

The above `segments` object is a list containing each found patch of the sosnovski plant. The coordinates are normalized to the original image size. 

An example patch is shown below. 

```python
[
    array([[   0.009375,      0.2375],
        [   0.009375,     0.26875],
        [   0.023438,     0.26875],
        [   0.023438,      0.2625],
        [      0.025,     0.26042],
        [      0.025,     0.25833],
        [   0.026563,     0.25625],
        [   0.032812,     0.25625],
        [   0.032812,      0.2375]], dtype=float32)
]
```

The first coordinate is the x-axis and the second coordinate is the y-axis. To plot on the image, we need to multiply the coordinates by the original image size. 

In [ ]:
# Converting to coordinates in pixels on the given image
segments_pixels = []
for segment in segments:
    segment_pixels = [(x * img_w, y * img_h) for x, y in segment]
    segments_pixels.append(segment_pixels)

In [ ]:
fig, ax = plt.subplots(1)
ax.imshow(img)

# Create and add a Polygon patch for each segment with red fill and border
for segment in segments_pixels:
    # Setting facecolor to red fills the polygon, and alpha adds transparency
    polygon = patches.Polygon(segment, closed=True, edgecolor='red', facecolor='red', linewidth=4, alpha=0.35)
    ax.add_patch(polygon)

plt.axis('off')
plt.show()

# Prediction to GPS conversion 

To get all the exif information, please visit: https://exif.tools/

In [ ]:
# Extracting the function to extract the focal length
def extract_focal_length(file_path):
    # Open the image file for reading in binary mode
    with open(file_path, 'rb') as f:
        # Read the EXIF data
        tags = exifread.process_file(f)

        # Check if GPSInfo tag is present
        if 'EXIF FocalLength' in tags:
            # Extract latitude, longitude, and altitude
            focal_length = tags['EXIF FocalLength'].values[0]

            return float(focal_length)
        else:
            print("Focal length not found in the metadata.")
            return None

In [ ]:
focal_length = extract_focal_length(path_to_img)
#focal_length = 4.5
print(f"The focal length is: {focal_length}mm")

In [ ]:
def extract_gps_coordinates(file_path):
    # Open the image file for reading in binary mode
    with open(file_path, 'rb') as f:
        # Read the EXIF data
        tags = exifread.process_file(f)

        # Check if GPSInfo tag is present
        if 'GPS GPSLatitude' in tags and 'GPS GPSLongitude' in tags and 'GPS GPSAltitude' in tags:
            # Extract latitude, longitude, and altitude
            latitude = tags['GPS GPSLatitude'].values
            longitude = tags['GPS GPSLongitude'].values
            altitude = tags['GPS GPSAltitude'].values

            # Convert coordinates to decimal format
            latitude_decimal = latitude[0] + latitude[1] / 60 + latitude[2] / 3600
            longitude_decimal = longitude[0] + longitude[1] / 60 + longitude[2] / 3600

            # 
            return float(latitude_decimal), float(longitude_decimal), float(altitude[0])
        else:
            print("GPS information not found in the metadata.")
            return None, None, None 

In [ ]:
# Extracting the metadata
latitude, longitude, altitude = extract_gps_coordinates(path_to_img)
print(f"The center coords are: {latitude}, {longitude}; Altitude {altitude}m")

In [ ]:
# Defining the camera properties (in mm)
# For the DJI Mavic 2 please visti: https://leapingwing.co.uk/gsd-calculator/
sensor_width = 6.4
sensor_height = 4.8

In [ ]:
# Hard saving other image related properties
relative_altitude = 50.46
gimbal_roll = 0
gimbal_yaw = 49.2
gimbal_pitch = -89.9

In [ ]:
# Defining the function that calculates the diff in pixels to meters
# GSD - Ground Sampling Distance
def calculate_gsd(
        height_from_ground, 
        image_width, 
        image_height, 
        sensor_width, 
        sensor_height, 
        focal_length
        ):
    """
    Function that calculates the GSD (Ground Sampling Distance) from pixels to meters

    Args:
        height_from_ground (float): Height from ground in meters
        image_width (int): Image width in pixels
        image_height (int): Image height in pixels
        sensor_width (float): Sensor width in mm
        sensor_height (float): Sensor height in mm
        focal_length (float): Focal length in mm

    Returns:
        gsd_h (float): Horizontal GSD in meters
        gsd_v (float): Vertical GSD in meters
    """
    # Calculating the horizontal and vertical GSD
    gsd_h = (height_from_ground * sensor_width) / (focal_length * image_width)
    gsd_v = (height_from_ground * sensor_height) / (focal_length * image_height)
    # Returning the average GSD
    return gsd_h, gsd_v

In [ ]:
# Calculating the diff in horizontal and vertical pixels to meters
gsd_h, gsd_v = calculate_gsd(
    relative_altitude,
    img.shape[1],
    img.shape[0],
    sensor_width, 
    sensor_height, 
    focal_length
    )

print(f"1 pixel in horizontal direction is {gsd_h} meters")
print(f"1 pixel in vertical direction is {gsd_v} meters")

In [ ]:
def pixel_to_gps_with_gimbal(x, y, width, height, lat_center, lon_center, gsd_h, gsd_v, yaw, pitch, roll, altitude):
    """
    Convert pixel coordinates to GPS coordinates considering gimbal orientation.
    
    Parameters:
    x, y          : Pixel coordinates
    width, height : Dimensions of the image
    lat_center    : Latitude of the center point (degrees)
    lon_center    : Longitude of the center point (degrees)
    gsd_h, gsd_v  : Horizontal and vertical GSD (meters/pixel)
    yaw, pitch, roll : Gimbal orientation (degrees)
    altitude      : Camera altitude (meters)
    
    Returns:
    (lat, lon) : GPS coordinates of the pixel
    """
    # Convert gimbal angles to radians
    yaw = math.radians(yaw)
    pitch = math.radians(pitch)
    roll = math.radians(roll)

    # Rotation matrices
    R_yaw = np.array([
        [math.cos(yaw), -math.sin(yaw), 0],
        [math.sin(yaw), math.cos(yaw), 0],
        [0, 0, 1]
    ])

    R_pitch = np.array([
        [math.cos(pitch), 0, math.sin(pitch)],
        [0, 1, 0],
        [-math.sin(pitch), 0, math.cos(pitch)]
    ])

    R_roll = np.array([
        [1, 0, 0],
        [0, math.cos(roll), -math.sin(roll)],
        [0, math.sin(roll), math.cos(roll)]
    ])

    # Combined rotation matrix
    R = R_yaw @ R_pitch @ R_roll

    # Offset from center in pixels
    dx = x - width / 2
    dy = y - height / 2

    # Real-world offsets in meters
    delta_x_real = dx * gsd_h
    delta_y_real = dy * gsd_v

    # Camera to world transformation
    ray_camera = np.array([delta_x_real, delta_y_real, altitude])
    ray_world = R @ ray_camera

    # Project onto ground plane (Z_w = 0)
    scale = altitude / ray_world[2]
    ground_offset = ray_world[:2] * scale

    # Convert offsets to GPS
    R_earth = 6371000  # Earth radius in meters
    delta_lat = ground_offset[1] / R_earth * (180 / math.pi)
    delta_lon = ground_offset[0] / (R_earth * math.cos(math.radians(lat_center))) * (180 / math.pi)

    # GPS coordinates
    lat = lat_center + delta_lat
    lon = lon_center + delta_lon

    return lat, lon

In [ ]:
# Creating the gps coordinates
polygon_points = results[0].masks[0].xy[0]
gps_points = []
for point in polygon_points:
    gps_points.append(
        pixel_to_gps_with_gimbal(
            point[0], 
            point[1], 
            img.shape[1], 
            img.shape[0], 
            latitude, 
            longitude, 
            gsd_h, 
            gsd_v, 
            gimbal_yaw, 
            gimbal_pitch, 
            gimbal_roll, 
            relative_altitude
            )
    )
    

In [ ]:
# Creating a dataframe out of the gps points
df = pd.DataFrame(gps_points, columns=["Latitude", "Longitude"])

# Saving to the inference directory 
df.to_csv(path_to_inference + "/" + base_name.split(".")[0] + ".csv", index=False)

In [ ]:
gps_points

# Sosnovski path prediction 

In [ ]:
def shift_polygon(polygon, wind_direction, displacement):
    """
    Shift a polygon based on wind direction and displacement.
    
    Parameters:
    polygon         : List of tuples (lat, lon) representing the polygon vertices
    wind_direction  : Wind direction in degrees (relative to true north)
    displacement    : Displacement caused by wind (in meters)
    
    Returns:
    shifted_polygon : List of tuples (lat, lon) representing the shifted polygon vertices
    """
    R = 6371000  # Earth's radius in meters
    
    # Convert wind direction to radians
    theta = math.radians(wind_direction)
    
    # Compute displacement in meters
    delta_x = displacement * math.cos(theta)  # Eastward displacement
    delta_y = displacement * math.sin(theta)  # Northward displacement
    
    shifted_polygon = []
    
    for lat, lon in polygon:
        # Compute latitude offset
        delta_lat = delta_y / R * (180 / math.pi)
        
        # Compute longitude offset (adjust for latitude)
        delta_lon = delta_x / (R * math.cos(math.radians(lat))) * (180 / math.pi)
        
        # Compute new GPS coordinates
        new_lat = lat + delta_lat
        new_lon = lon + delta_lon
        
        shifted_polygon.append((new_lat, new_lon))
    
    return shifted_polygon

In [ ]:
wind_direction = 90  # Wind blowing east
displacement = 10  # Wind displacement in meters

shifted_polygon = shift_polygon(gps_points, wind_direction, displacement)

In [ ]:
# Saving into a dataframe
df_shifted = pd.DataFrame(shifted_polygon, columns=["Latitude", "Longitude"])
df_shifted.to_csv(path_to_inference + "/" + base_name.split(".")[0] + "_shifted.csv", index=False)